In [1]:
# !pip install papermill

In [2]:
from datetime import datetime
import json

def now():
    now = datetime.now()
    yyyymmdd_hhmmss_part = now.strftime('%Y-%m-%d %H:%M:%S')
    ms_part = f'{int(now.microsecond / 1000):03d}'
    return f'{yyyymmdd_hhmmss_part},{ms_part}'

def load_results(results_filename):
    with open(results_filename, 'r', encoding='utf-8') as f:
        content = json.load(f)
        f1_weighted = content["leaderboard"][0]["score_val"]
        fit_time = content["leaderboard"][0]["fit_time"]
    return f'{f1_weighted:.6f}', f'{fit_time:.3f}'

In [3]:
from pathlib import Path
from time import sleep
from tqdm.notebook import tqdm
import os
import papermill as pm

errors = {}

base_time = 60

# for kind in tqdm(["Binary", "Multiclass"], desc='Kind', leave='False'):
for kind in tqdm(["Multiclass", "Binary"], desc='Kind', leave='False'):

    base_folder = Path(f"2025-06-13/{kind}")
    
    # Find all .parquet files recursively
    parquet_files = list(base_folder.rglob("*.parquet"))
    
    # Sort by file size (ascending)
    parquet_files_sorted = sorted(parquet_files, key=lambda p: os.path.getsize(p))
    
    # Iterate
    for file in tqdm(parquet_files_sorted, desc='File', leave='False'):
        
        base_filename = os.path.join(file.parent, file.stem)
        results_filename = f'{base_filename}_results.json'
        size_mb = f'{(os.path.getsize(file) / (1024 * 1024)):.3f} MB'
        input_notebook = 'ag_code.ipynb'
        output_notebook = f'{base_filename}.ipynb'
        
        max_time = base_time
        success = False

        while not success and max_time <= 3840:
        
            try:

                # if 'NIDS_NF-UQ-NIDS-v2' in str(file) or 'IoT_23' in str(file) or 'KDD_Cup_1999' in str(file):
                #     break
                # if 'ToN_IoT' not in str(file):
                #     break
        
                tqdm.write(f'[{now()}] Processing | FILE = {str(file.stem):<64} | FILE_SIZE   = {size_mb:<12} | MAX_TIME = {max_time:<6}')
            
                if not Path(results_filename).exists():
    
                    pm.execute_notebook(
                        input_notebook,
                        output_notebook,
                        parameters=dict(
                            BASE_FILENAME=base_filename,
                            TARGET_COL='label',
                            TEST_SIZE=0.2,
                            TIME_LIMIT=max_time,
                            EVAL_METRIC='f1_weighted',
                            PRESET='medium_quality',
                            N_FOLDS=5,
                            RANDOM_STATE=42
                        )
                    )
            
                f1_weighted, fit_time = load_results(results_filename)
                tqdm.write(f'[{now()}] Processed  | FILE = {str(file.stem):<64} | F1_WEIGHTED = {f1_weighted:<12} | FIT_TIME = {fit_time:<6}')
                success = True
                
            except Exception as e:
                # if 'Consider specifying a larger time_limit.' in str(e):
                if 'raise_on_no_models_fitted' in str(e) or 'time_limit' in str(e):
                    max_time = 2 * max_time
                else:
                    tqdm.write(f'[{now()}] ERROR      | FILE = {str(file.stem):<64} | EXCEPTION = {str(e)}')
                    errors[str(file.stem)] = e
                    break

Kind:   0%|          | 0/2 [00:00<?, ?it/s]

File:   0%|          | 0/61 [00:00<?, ?it/s]

[2025-06-21 02:12:59,127] Processing | FILE = ToN_IoT_IoT_Garage_Door_Multiclass                               | FILE_SIZE   = 0.003 MB     | MAX_TIME = 60    


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

[2025-06-21 02:13:02,001] ERROR      | FILE = ToN_IoT_IoT_Garage_Door_Multiclass                               | EXCEPTION = 
---------------------------------------------------------------------------
Exception encountered at "In [3]":
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[3], line 21
     18 from autogluon.tabular import TabularDataset
     19 from sklearn.model_selection import train_test_split
---> 21 train_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=df[TARGET_COL])
     23 train_data, test_data = TabularDataset(train_df), TabularDataset(test_df)
     25 # Step 4: Print dtypes

File ~/miniforge3/envs/ag/lib/python3.11/site-packages/sklearn/utils/_param_validation.py:216, in validate_params.<locals>.decorator.<locals>.wrapper(*args, **kwargs)
    210 try:
    211     with config_context(
    212         skip_paramet

...

Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

[2025-06-21 03:37:46,808] Processing | FILE = NIDS_NF-UQ-NIDS-v2_Binary                                        | FILE_SIZE   = 1816.803 MB  | MAX_TIME = 240   


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

*** SIGTERM received at time=1750490818 on cpu 2 ***
PC: @     0x71dd4a925e2e  (unknown)  epoll_wait
    @     0x71db90bac0ed         64  absl::lts_20230802::AbslFailureSignalHandler()
    @     0x71dd4a842520  (unknown)  (unknown)
[2025-06-21 04:26:58,687 E 2455 2455] logging.cc:440: *** SIGTERM received at time=1750490818 on cpu 2 ***
[2025-06-21 04:26:58,687 E 2455 2455] logging.cc:440: PC: @     0x71dd4a925e2e  (unknown)  epoll_wait
[2025-06-21 04:26:58,689 E 2455 2455] logging.cc:440:     @     0x71db90bac119         64  absl::lts_20230802::AbslFailureSignalHandler()
[2025-06-21 04:26:58,689 E 2455 2455] logging.cc:440:     @     0x71dd4a842520  (unknown)  (unknown)


[2025-06-21 04:27:01,504] Processed  | FILE = NIDS_NF-UQ-NIDS-v2_Binary                                        | F1_WEIGHTED = 0.551050     | FIT_TIME = 179.600


In [4]:
from pprint import pprint

pprint(errors, indent=4)

{   'BCCC_CIC-BCCC-NRC-IoT-2023-Original_Training_and_Testing_Binary': PapermillExecutionError(2, 3, 'import json\nimport pandas as pd\n\npd.set_option("display.max_rows", None)\npd.set_option("display.max_columns", None)\n\n# Step 1: Load parquet and metadata\ndf = pd.read_parquet(f\'{BASE_FILENAME}.parquet\')#.dropna()\n\nwith open(f\'{BASE_FILENAME}.json\', \'r\', encoding=\'utf-8\') as f:\n    metadata = json.load(f)\n\n# Step 2: Apply dtypes from metadata to df\nfor col, dtype in metadata["dtypes"].items():\n    df[col] = df[col].astype(dtype)\n\n# Step 3: Split the data into 80% train (for CV) and 20% test (holdout)\nfrom autogluon.tabular import TabularDataset\nfrom sklearn.model_selection import train_test_split\n\ntrain_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=df[TARGET_COL])\n\ntrain_data, test_data = TabularDataset(train_df), TabularDataset(test_df)\n\n# Step 4: Print dtypes\nprint(f\'{"column":<40}\', f\'{"metadata":<20}\',